## 54개 군집

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.cluster import KMeans
import folium
import numpy as np

# 1. Shapefile을 사용해 세종시 동 경계선 불러오기
shp_file_path = '/Users/jiminsim/Desktop/gong/BND_ADM_DONG_PG.shp'
sejong_geo = gpd.read_file(shp_file_path, encoding='euc-kr')

# 2. 좌표계를 EPSG:4326으로 변환
if sejong_geo.crs != "EPSG:4326":
    sejong_geo = sejong_geo.to_crs(epsg=4326)

# 3. 필터링할 동 코드 목록 정의
target_dong_codes = ['29010110', '29010512', '29010513', '29010514', '29010515', 
                     '29010521', '29010522', '29010560', '29010590', '29010600', 
                     '29010610', '29010640', '29010660', '29010680', '29010710']

# 4. 선택한 동 필터링
filtered_dong = sejong_geo[sejong_geo['ADM_CD'].isin(target_dong_codes)]

# 5. 데이터 불러오기 (정규화된 데이터)
bus_data = pd.read_csv('/Users/jiminsim/Downloads/bus_total_filtered_corrected.csv', encoding='utf-8-sig')
tt5 = pd.read_csv('/Users/jiminsim/Downloads/filtered_grid_points_with_total.csv')
sj_sch3 = pd.read_csv('/Users/jiminsim/Downloads/sj_sch3_filtered_corrected.csv', encoding='utf-8-sig')

# 필요한 정규화된 컬럼 추출하여 병합 (정규화된 값으로 지점 선정)
bus_coords = bus_data[['위도', '경도', '합계_정규화']].rename(columns={'합계_정규화': '이용객수'})
tt5_coords = tt5[['위도', '경도', '종사자수_정규화']].rename(columns={'종사자수_정규화': '이용객수'})
school_coords = sj_sch3[['위도', '경도', '총계_정규화']].rename(columns={'총계_정규화': '이용객수'})

# 데이터 병합
data = pd.concat([bus_coords, tt5_coords, school_coords], ignore_index=True)

# 6. NaN 값 처리
data = data.fillna(0)

# ---------------- K-Means 군집화 실행 (군집 수 54개) ---------------- #
n_clusters = 54  # 군집 수 설정
weights = data['이용객수'].values  # 가중치 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(data[['위도', '경도']], sample_weight=weights)

# 군집 중심 추출
cluster_centers = kmeans.cluster_centers_
cluster_data = pd.DataFrame(cluster_centers, columns=['위도', '경도'])
cluster_data['군집'] = np.arange(len(cluster_centers))

# ---------------- Folium 지도 생성 및 시각화 ---------------- #
sejong_center = [36.4872, 127.2817]
map_kmeans = folium.Map(location=sejong_center, zoom_start=12, tiles="cartodbpositron")

# 동 경계선 추가 (회색 스타일 적용)
folium.GeoJson(
    filtered_dong.to_json(),
    name="동 경계선",
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 2
    }
).add_to(map_kmeans)

# 각 클러스터 중심에 초록색 마커 추가
for i, row in cluster_data.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='green'),  # 초록색 마커
        popup=f"K-Means 클러스터 중심: 군집 {row['군집']}"
    ).add_to(map_kmeans)

# 지도 저장
map_kmeans.save('/Users/jiminsim/Desktop/sejong_kmeans_final_clusters_with_green_markers.html')

# 지도 확인
map_kmeans


## 54개 군집 중 커버 인구수가 많은 상위 15개 군집

In [41]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.cluster import KMeans
import folium
import numpy as np
from geopy.distance import geodesic

# 1. Shapefile을 사용해 세종시 동 경계선 불러오기
shp_file_path = '/Users/jiminsim/Desktop/gong/BND_ADM_DONG_PG.shp'
sejong_geo = gpd.read_file(shp_file_path, encoding='euc-kr')

# 2. 좌표계를 EPSG:4326으로 변환 (WGS84)
if sejong_geo.crs != "EPSG:4326":
    sejong_geo = sejong_geo.to_crs(epsg=4326)

# 3. 필터링할 동 코드 목록 정의
target_dong_codes = ['29010110', '29010512', '29010513', '29010514', '29010515', 
                     '29010521', '29010522', '29010560', '29010590', '29010600', 
                     '29010610', '29010640', '29010660', '29010680', '29010710']

# 4. 선택한 동 필터링
filtered_dong = sejong_geo[sejong_geo['ADM_CD'].isin(target_dong_codes)]

# 5. 데이터 불러오기 (정규화된 데이터와 정규화 전 데이터)
bus_data = pd.read_csv('/Users/jiminsim/Downloads/bus_total_filtered_corrected.csv', encoding='utf-8-sig')
tt5 = pd.read_csv('/Users/jiminsim/Downloads/filtered_grid_points_with_total.csv')
sj_sch3 = pd.read_csv('/Users/jiminsim/Downloads/sj_sch3_filtered_corrected.csv', encoding='utf-8-sig')

# 정규화된 데이터 사용 (K-Means 가중치 군집화를 위한 데이터)
bus_coords = bus_data[['위도', '경도', '합계_정규화']].rename(columns={'합계_정규화': '이용객수'})
tt5_coords = tt5[['위도', '경도', '종사자수_정규화']].rename(columns={'종사자수_정규화': '이용객수'})
school_coords = sj_sch3[['위도', '경도', '총계_정규화']].rename(columns={'총계_정규화': '이용객수'})

# 정규화된 데이터를 병합하여 K-Means 군집화에 사용
data = pd.concat([bus_coords, tt5_coords, school_coords], ignore_index=True)

# 정규화 전 데이터 사용 (커버 인구수 계산을 위한 데이터)
bus_coords_orig = bus_data[['위도', '경도', '합계']].rename(columns={'합계': '이용객수'})
tt5_coords_orig = tt5[['위도', '경도', '종사자수_나눈값']].rename(columns={'종사자수_나눈값': '이용객수'})
school_coords_orig = sj_sch3[['위도', '경도', '총계']].rename(columns={'총계': '이용객수'})

# 정규화 전 데이터 병합 (커버 인구수 계산용)
data_orig = pd.concat([bus_coords_orig, tt5_coords_orig, school_coords_orig], ignore_index=True)

# 6. NaN 값 처리
data = data.fillna(0)
data_orig = data_orig.fillna(0)

# ---------------- K-Means 군집화 실행 (군집 수 54개, 정규화된 데이터 사용) ---------------- #
n_clusters = 54  # 군집 수 설정
weights = data['이용객수'].values  # 가중치 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(data[['위도', '경도']], sample_weight=weights)

# 군집 중심 추출
cluster_centers = kmeans.cluster_centers_
cluster_data = pd.DataFrame(cluster_centers, columns=['위도', '경도'])
cluster_data['군집'] = np.arange(len(cluster_centers))

# ---------------- 반경 313.5m 내 커버 인구수 계산 함수 (하버사인 거리 사용) ---------------- #
def calculate_covered_population(center, radius, data_orig):
    """하버사인 거리를 사용하여 중심 좌표와 반경 내에 있는 좌표들의 이용객수 합산 (중복 카운트 방지)"""
    total_population = 0
    visited_points = set()  # 중복된 좌표 카운트를 방지하기 위한 집합

    for idx, row in data_orig.iterrows():
        point = (row['위도'], row['경도'])
        distance = geodesic(center, point).meters  # 중심과 각 좌표의 거리 계산 (미터 단위)

        if distance <= radius and point not in visited_points:
            total_population += row['이용객수']
            visited_points.add(point)  # 중복 방지

    return total_population

# 각 군집의 커버 인구수 계산
cluster_data['커버 인구수'] = cluster_data.apply(lambda row: calculate_covered_population([row['위도'], row['경도']], 313.5, data_orig), axis=1)

# 커버 인구수가 가장 높은 상위 15개 군집 추출
top_15_clusters = cluster_data.nlargest(15, '커버 인구수')

# 상위 15개 군집의 커버 인구수를 출력
top_15_clusters_sorted = top_15_clusters[['군집', '위도', '경도', '커버 인구수']].sort_values(by='커버 인구수', ascending=False)
print("상위 15개 군집이 커버하는 인구수 (내림차순):")
print(top_15_clusters_sorted.to_string(index=False))

# ---------------- Folium 지도 생성 및 시각화 (상위 15개 군집) ---------------- #
map_kmeans_top15 = folium.Map(location=[36.4872, 127.2817], zoom_start=12)

# 동 경계선 추가 (회색 스타일 적용)
folium.GeoJson(
    filtered_dong.to_json(),
    name="동 경계선",
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 2
    }
).add_to(map_kmeans_top15)

# 상위 15개 군집에 대해 마커와 반경 313.5m 원 추가 + 인구수 표시
for i, row in top_15_clusters.iterrows():
    # 중심 좌표
    center = [row['위도'], row['경도']]
    
    # 팝업 내용 정의
    popup_text = f"군집 번호: {row['군집']}<br>커버 인구수: {row['커버 인구수']}"
    popup = folium.Popup(popup_text, max_width=300)

    # 마커 추가 (팝업에 군집 번호와 커버 인구수를 표시)
    folium.Marker(
        location=center,
        radius=3,  # 마커 크기
        color='green',  # 마커 색상 (상위 군집 표시를 위해 초록색 사용)
        fill=True,
        fill_opacity=0.7,
        popup=popup  # 팝업 추가
    ).add_to(map_kmeans_top15)

    # 반경 313.5m 원 추가
    folium.Circle(
        location=center,
        radius=313.5,  # 실제로 313.5m 반경
        color='green',
        fill=True,
        fill_opacity=0.2
    ).add_to(map_kmeans_top15)

# 지도 저장
map_kmeans_top15.save('/Users/jiminsim/Desktop/sejong_kmeans_top_15_clusters_with_population.html')

# 지도 확인
map_kmeans_top15


상위 15개 군집이 커버하는 인구수 (내림차순):
 군집        위도         경도       커버 인구수
 46 36.601859 127.300972 15061.000000
 34 36.504596 127.261548 10383.926341
 30 36.499723 127.265752  9254.703414
 42 36.485624 127.250890  8854.695716
  5 36.500925 127.260301  8835.102439
  2 36.493710 127.303119  8677.040000
  1 36.600709 127.297002  8223.030000
 33 36.489174 127.263892  8165.087145
 28 36.610509 127.286545  7501.000000
 12 36.504863 127.267115  7308.988536
 38 36.518942 127.233957  5335.562631
 31 36.491413 127.256792  5109.225716
 16 36.480660 127.289021  4982.320000
 47 36.478257 127.288014  4774.870000
 26 36.502963 127.248972  4747.190000


## 각 군집 커버 인구수

In [47]:
# ---------------- 각 군집의 커버 인구수 계산 및 저장 ---------------- #
cluster_data['커버 인구수'] = cluster_data.apply(lambda row: calculate_covered_population([row['위도'], row['경도']], 313.5, data_orig), axis=1)

# 커버 인구수를 내림차순으로 정렬하여 군집 번호 출력
cluster_data_sorted = cluster_data.sort_values(by='커버 인구수', ascending=False)

print("커버 인구수 많은 순서대로 군집 번호:")
for i, row in cluster_data_sorted.iterrows():
    print(f"군집 {row['군집']}의 커버 인구수: {row['커버 인구수']}")


커버 인구수 많은 순서대로 군집 번호:
군집 46.0의 커버 인구수: 15061.0
군집 34.0의 커버 인구수: 10383.9263412
군집 30.0의 커버 인구수: 9254.7034144
군집 42.0의 커버 인구수: 8854.695716
군집 5.0의 커버 인구수: 8835.102438800002
군집 2.0의 커버 인구수: 8677.04
군집 1.0의 커버 인구수: 8223.029999999999
군집 33.0의 커버 인구수: 8165.0871449999995
군집 28.0의 커버 인구수: 7501.0
군집 12.0의 커버 인구수: 7308.988536400002
군집 38.0의 커버 인구수: 5335.562631200001
군집 31.0의 커버 인구수: 5109.225716000001
군집 16.0의 커버 인구수: 4982.320000000002
군집 47.0의 커버 인구수: 4774.870000000002
군집 26.0의 커버 인구수: 4747.1900000000005
군집 0.0의 커버 인구수: 4737.030000000001
군집 40.0의 커버 인구수: 4548.98
군집 23.0의 커버 인구수: 4481.5599999999995
군집 22.0의 커버 인구수: 3360.33
군집 17.0의 커버 인구수: 3305.4300000000003
군집 18.0의 커버 인구수: 3276.88
군집 35.0의 커버 인구수: 3252.58
군집 7.0의 커버 인구수: 3146.14
군집 9.0의 커버 인구수: 2623.422858
군집 25.0의 커버 인구수: 2265.6873684
군집 29.0의 커버 인구수: 1984.84
군집 8.0의 커버 인구수: 1788.37
군집 15.0의 커버 인구수: 1623.17
군집 32.0의 커버 인구수: 1556.7
군집 53.0의 커버 인구수: 1508.8
군집 36.0의 커버 인구수: 1482.65
군집 19.0의 커버 인구수: 1401.645
군집 43.0의 커버 인구수: 1225.19
군집 4.0의 커버 인구수